<a href="https://colab.research.google.com/github/nyongja/fastcamput-recsys/blob/main/01_%EC%BB%A8%ED%85%90%EC%B8%A0%EA%B8%B0%EB%B0%98_06_TF_IDF_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- kss는 한국어 문장 분리 모듈로, 자세한 내용은 [링크](https://pypi.org/project/kss/)를 참고하세요.

In [2]:
!pip install kss

     |████████████████████████████████| 71kB 3.4MB/s 


In [3]:
import os
import kss
import re

In [4]:
# 형태소 기반 토크나이징 (Konlpy)
!python3 -m pip install konlpy
# mecab (ubuntu, mac 기준)
# 다른 os 설치 방법 및 자세한 내용은 다음 참고: https://konlpy.org/ko/latest/install/#id1
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 460kB 50.6MB/s 
     |████████████████████████████████| 92kB 9.9MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
Installing automake (A dependency for mecab-ko)
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.

### Read dataset
- Github의 data 디렉토리에서 news_sample.txt 파일을 다운받아서 분석에 사용해주세요.

In [5]:
removal_list =  "‘, ’, ◇, ‘, ”,  ’, ', ·, \“, ·, △, ●,  , ■, (, ), \", >>, `, /, -,∼,=,ㆍ<,>, .,?, !,【,】, …, ◆,%"

def cleansing_special(sentence):
    # 특수문자를 전처리를 하는 함수
    sentence = re.sub("[.,\'\"’‘”“!?]", "", sentence)
    sentence = re.sub("[^가-힣0-9a-zA-Z\\s]", " ", sentence)
    sentence = re.sub("\s+", " ", sentence)
    
    sentence = sentence.translate(str.maketrans(removal_list, ' '*len(removal_list)))
    sentence = sentence.strip()
    
    return sentence

def cleansing_numbers(sentence):
    # 숫자를 전처리(delexicalization) 하는 함수    
    sentence = re.sub('[0-9]+', 'NUM', sentence)
    sentence = re.sub('NUM\s+', "NUM", sentence)
    sentence = re.sub('[NUM]+', "NUM", sentence)
    
    return sentence


In [6]:
from konlpy.tag import Mecab
mecab = Mecab()

In [13]:
# 각자 작업 환경에 맞는 경로를 지정해주세요. Google Colab과 Jupyter환경에서 경로가 다를 수 있습니다.
data_path = '/content/drive/MyDrive/Colab Notebooks/fast-campus/data/'
all_sentences = []
with open(os.path.join(data_path, 'news_sample.txt'), 'r', encoding='utf-8') as f:
  for idx, line in enumerate(f.readlines()):
    print(f"--- 문서 {idx} 번 ---")
    preprocessed = cleansing_numbers(line)
    preprocessed = cleansing_special(preprocessed)
    # 명사만 추출하기 - 은는이가 부분은 필요없으니까
    preprocessed_news = ' '.join(list(set(mecab.nouns(preprocessed))))
    # 문장 분리해서 사용하는 방법
    # preprocessed_news = ' '.join(kss.split_sentences(preprocessed))
    print(preprocessed_news)
    all_sentences.append(preprocessed_news)

--- 문서 0 번 ---
이 정치력 선거 출발 속 민족 육 번 유치 문제 추진 새해 욕 기억 바탕 협의 기 모색 자질 미래 포함 달성 책임 듯 표출 정립 저력 불씨 정부 한뜻 도약 추진력 남북 시 관계 한미 퇴행 단계 일 조기 선진국 발전 역량 파악 강원도 중요 단절 발휘 코드 당국 지 인구 정책 미국 극대 후보 단체장 여건 지혜 통일 년 평창 해결 현안 동해 북 대비 화두 청사진 얼마 단체 암투 경제 결정 증오 근거 위기 외교 음모 기대 낭비성 열차 다음 바람 아침 한반도 쇠 운행 경기 등 공유 경의선 올해 남북한 회생 통행 도전 몫 직무 지난해 마음 정상 오바마 생산 개성관광 관행 점 개성 기초자치단체 변화 틀 중단 성향 공직자 우리 정과 갈등 인맥 국민 짐 능력 현상 방향 불신 경협 인재 국호 과정 변수 행정력 태양 그동안 화해 면적 인원 청산 성과 피 사무소 성공 광채 역사 취임 암울 자신 좌절 시대 유념 때 소망 다이 고유 대화 올림픽 유능 휘하 원년 축년 슬기 자생력 목적 민 도민 배치 국익 번지 이것 흑인 방법 뒤 도정 공감대 절망 동계 지방 사람 정치 도시 부여 나열 기축 날 기회비용 국내 꿈 한국 정치색 행정 활력 대통령 구축 실행 인사 평화 수 시간 자치 것 발굴 단추 최대 약점 무엇 적재적소 전환점 의 회한 희망 동기 예산 열쇠 핵 도 가치 전력 폐쇄 생명력 련 집행 혼란 극복 논리 떼 당위 땅 시련 질주 자본 업그레이드 해다 몸 고난 조화 군 때문 바다 건국 처음 후 대결 금강산 지역 구체 미증유 기업 글로벌 과제 햇살 활용 간 기반 어제
--- 문서 1 번 ---
이 선수 도출 육 월 새해 회장 과시 바탕 협의 기 체육관 라인 연구 시책 모습 준수 보조 우수 무대 향상 각오 봉사 저력 반영 시 육성 강도 팀 단계 일 시스템 복지 폭 사회 역량 발전 발휘 동물 지금 사항 공조 정책 안 단체장 근면 소통 행복 기원 년 내실 혼신 사업 운영 걸음 개정 임직원 단체 베이징 경제 성원 양 도종 수동 기대 규정 이사 특성 강원 협조 역동 육회 경

### Scikit-learn을 사용한 TF-IDF

- 한글자 단어 제외
- https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
- Useful option: tokenizer, stop_words



In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

In [15]:
print(len(all_sentences))

20


In [19]:
print(' '.join(all_sentences))

이 정치력 선거 출발 속 민족 육 번 유치 문제 추진 새해 욕 기억 바탕 협의 기 모색 자질 미래 포함 달성 책임 듯 표출 정립 저력 불씨 정부 한뜻 도약 추진력 남북 시 관계 한미 퇴행 단계 일 조기 선진국 발전 역량 파악 강원도 중요 단절 발휘 코드 당국 지 인구 정책 미국 극대 후보 단체장 여건 지혜 통일 년 평창 해결 현안 동해 북 대비 화두 청사진 얼마 단체 암투 경제 결정 증오 근거 위기 외교 음모 기대 낭비성 열차 다음 바람 아침 한반도 쇠 운행 경기 등 공유 경의선 올해 남북한 회생 통행 도전 몫 직무 지난해 마음 정상 오바마 생산 개성관광 관행 점 개성 기초자치단체 변화 틀 중단 성향 공직자 우리 정과 갈등 인맥 국민 짐 능력 현상 방향 불신 경협 인재 국호 과정 변수 행정력 태양 그동안 화해 면적 인원 청산 성과 피 사무소 성공 광채 역사 취임 암울 자신 좌절 시대 유념 때 소망 다이 고유 대화 올림픽 유능 휘하 원년 축년 슬기 자생력 목적 민 도민 배치 국익 번지 이것 흑인 방법 뒤 도정 공감대 절망 동계 지방 사람 정치 도시 부여 나열 기축 날 기회비용 국내 꿈 한국 정치색 행정 활력 대통령 구축 실행 인사 평화 수 시간 자치 것 발굴 단추 최대 약점 무엇 적재적소 전환점 의 회한 희망 동기 예산 열쇠 핵 도 가치 전력 폐쇄 생명력 련 집행 혼란 극복 논리 떼 당위 땅 시련 질주 자본 업그레이드 해다 몸 고난 조화 군 때문 바다 건국 처음 후 대결 금강산 지역 구체 미증유 기업 글로벌 과제 햇살 활용 간 기반 어제 이 선수 도출 육 월 새해 회장 과시 바탕 협의 기 체육관 라인 연구 시책 모습 준수 보조 우수 무대 향상 각오 봉사 저력 반영 시 육성 강도 팀 단계 일 시스템 복지 폭 사회 역량 발전 발휘 동물 지금 사항 공조 정책 안 단체장 근면 소통 행복 기원 년 내실 혼신 사업 운영 걸음 개정 임직원 단체 베이징 경제 성원 양 도종 수동 기대 규정 이사 특성 강원 협조 역동 육회 경기 취약 마련 등 올해 김진선지사 지난해 진통 최일선 

In [16]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_sentences)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(tfidf_vectorizer.get_feature_names()):
    word2id[feature] = idx

In [17]:
tfidf_vectorizer.vocabulary_

{'정치력': 1935,
 '선거': 1111,
 '출발': 2252,
 '민족': 802,
 '유치': 1617,
 '문제': 768,
 '추진': 2239,
 '새해': 1089,
 '기억': 362,
 '바탕': 817,
 '협의': 2542,
 '모색': 736,
 '자질': 1796,
 '미래': 781,
 '포함': 2402,
 '달성': 514,
 '책임': 2155,
 '표출': 2410,
 '정립': 1912,
 '저력': 1837,
 '불씨': 988,
 '정부': 1915,
 '한뜻': 2459,
 '도약': 593,
 '추진력': 2240,
 '남북': 419,
 '관계': 231,
 '한미': 2460,
 '퇴행': 2331,
 '단계': 496,
 '조기': 1957,
 '선진국': 1119,
 '발전': 844,
 '역량': 1413,
 '파악': 2359,
 '강원도': 59,
 '중요': 2030,
 '단절': 505,
 '발휘': 851,
 '코드': 2284,
 '당국': 519,
 '인구': 1708,
 '정책': 1932,
 '미국': 779,
 '극대': 325,
 '후보': 2618,
 '단체장': 509,
 '여건': 1397,
 '지혜': 2089,
 '통일': 2322,
 '평창': 2382,
 '해결': 2481,
 '현안': 2529,
 '동해': 633,
 '대비': 547,
 '화두': 2563,
 '청사진': 2176,
 '얼마': 1383,
 '단체': 508,
 '암투': 1344,
 '경제': 138,
 '결정': 122,
 '증오': 2043,
 '근거': 328,
 '위기': 1582,
 '외교': 1524,
 '음모': 1628,
 '기대': 352,
 '낭비성': 427,
 '열차': 1454,
 '다음': 492,
 '바람': 812,
 '아침': 1332,
 '한반도': 2461,
 '운행': 1559,
 '경기': 128,
 '공유': 201,
 '경의선': 135,
 '올해': 1515

In [ ]:
tfidf_matrix.shape

(20, 2630)

In [20]:
for idx, sent in enumerate(all_sentences):
    print(f"--- 문서 {idx} 번 ---")
    results = [(token, tfidf_matrix[idx, word2id[token]]) for token in sent.split()]
    results.sort(key = lambda element : element[1], reverse=True)
    print(results)
    print("\n")

--- 문서 0 번 ---
[('정치력', 0.0831600526056308), ('선거', 0.0831600526056308), ('기억', 0.0831600526056308), ('자질', 0.0831600526056308), ('정립', 0.0831600526056308), ('불씨', 0.0831600526056308), ('추진력', 0.0831600526056308), ('한미', 0.0831600526056308), ('퇴행', 0.0831600526056308), ('단절', 0.0831600526056308), ('당국', 0.0831600526056308), ('화두', 0.0831600526056308), ('청사진', 0.0831600526056308), ('암투', 0.0831600526056308), ('증오', 0.0831600526056308), ('근거', 0.0831600526056308), ('외교', 0.0831600526056308), ('음모', 0.0831600526056308), ('낭비성', 0.0831600526056308), ('열차', 0.0831600526056308), ('아침', 0.0831600526056308), ('한반도', 0.0831600526056308), ('운행', 0.0831600526056308), ('경의선', 0.0831600526056308), ('남북한', 0.0831600526056308), ('통행', 0.0831600526056308), ('직무', 0.0831600526056308), ('오바마', 0.0831600526056308), ('개성관광', 0.0831600526056308), ('관행', 0.0831600526056308), ('개성', 0.0831600526056308), ('기초자치단체', 0.0831600526056308), ('공직자', 0.0831600526056308), ('정과', 0.0831600526056308), ('인맥', 0.08316005